In [25]:
#input 336 output 96
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# random seed
parser.add_argument('--random_seed', type=int, default=2021, help='random seed')

# basic config
parser.add_argument('--is_training', type=int, default=1, help='status')
parser.add_argument('--model_id', type=str, default='test', help='model id')
parser.add_argument('--model', type=str, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str,  default='ETTh2', help='dataset type')
parser.add_argument('--root_path', type=str, default='/ssddata/data/jiahuili/PatchTST/all_six_datasets/ETT-small/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh2.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=336, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
#parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# PatchTST
parser.add_argument('--fc_dropout', type=float, default=0.0, help='fully connected dropout')
parser.add_argument('--head_dropout', type=float, default=0.0, help='head dropout')
parser.add_argument('--patch_len', type=int, default=16, help='patch length')
parser.add_argument('--stride', type=int, default=8, help='stride')
parser.add_argument('--padding_patch', default='end', help='None: None; end: padding on the end')
parser.add_argument('--revin', type=int, default=1, help='RevIN; True 1 False 0')
parser.add_argument('--affine', type=int, default=1, help='RevIN-affine; True 1 False 0')
parser.add_argument('--subtract_last', type=int, default=0, help='0: subtract mean; 1: subtract last')
parser.add_argument('--decomposition', type=int, default=0, help='decomposition; True 1 False 0')
parser.add_argument('--kernel_size', type=int, default=25, help='decomposition-kernel')
parser.add_argument('--individual', type=int, default=0, help='individual head; True 1 False 0')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=64, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=3, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=128, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.0, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=100, help='train epochs')
parser.add_argument('--batch_size', type=int, default=128, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=10, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type3', help='adjust learning rate')
parser.add_argument('--pct_start', type=float, default=0.3, help='pct_start')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

parser.add_argument('--scale', type=int, default=10, help='scale')
parser.add_argument('--gate',type=int,default=1)
parser.add_argument('--channel_dependent', type=int, default=0, help='gate')

args,unkown = parser.parse_known_args()

In [26]:
from models.MSPTST import Model
model=Model(args)

In [27]:
import torch
model = model.to('cuda:0')
model.load_state_dict(torch.load('/ssddata/data/jiahuili/PatchTST/checkpoints/336_96_MSPTST_ETTh2_ftM_sl336_ll0_pl96_dm64_nh8_el3_dl1_df128_fc1_ebtimeF_dtTrue_Exp_0/checkpoint.pth'))
model.eval()

Model(
  (model): MSPTST_backbone(
    (revin_layer): RevIN()
    (backbone): ModuleList(
      (0): TSTiEncoder(
        (W_P): Linear(in_features=16, out_features=64, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (encoder): TSTEncoder(
          (layers): ModuleList(
            (0): TSTEncoderLayer(
              (self_attn): _MultiheadAttention(
                (W_Q): Linear(in_features=64, out_features=64, bias=True)
                (W_K): Linear(in_features=64, out_features=64, bias=True)
                (W_V): Linear(in_features=64, out_features=64, bias=True)
                (sdp_attn): _ScaledDotProductAttention(
                  (attn_dropout): Dropout(p=0.0, inplace=False)
                )
                (to_out): Sequential(
                  (0): Linear(in_features=64, out_features=64, bias=True)
                  (1): Dropout(p=0.0, inplace=False)
                )
              )
              (dropout_attn): Dropout(p=0.0, inplace=False)
       

In [32]:
from data_provider.data_factory import data_provider
import matplotlib.pyplot as plt
from statistics import mean
import torch.nn as nn
test_data, test_loader = data_provider(args,flag='test')
%matplotlib inline
import numpy as np
total_MSE=[]
total_MAE=[]
MSE=nn.MSELoss()
MAE=nn.L1Loss()
ratio=0.9

preds=[]
trues=[]
with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
                # (batch_x, batch_y, batch_x_mark, batch_y_mark) =next(iter(test_loader))
                
                
                
                batch_x=batch_x.float().to('cuda:0')
                batch_y=batch_y.float().to('cuda:0')
                # print(batch_x.shape)
                mask=torch.rand(batch_x[:,:,:].shape) < ratio
                batch_x[:,:,:]=batch_x[:,:,:]*mask.to('cuda:0')
                outputs = model(batch_x,batch_x_mark,batch_y_mark)
                
                
                

                
                
                
                pred = outputs[:,:,:].detach().cpu().numpy()  # outputs.detach().cpu().numpy()  # .squeeze()
                true = batch_y[:,:,:].detach().cpu().numpy()  # batch_y.detach().cpu().numpy()  # .squeeze()
                
                preds.append(pred)
                trues.append(true)
                
                # total_MSE.append(mae)
preds = np.concatenate(preds, axis=0)
trues = np.concatenate(trues, axis=0)

mse=np.mean(np.abs(preds - trues))
print(mse)
# plt.plot(pred[3,:,4],label='pred')

# plt.legend()

test 2785
0.38284567
